In [5]:
%load_ext autoreload
%autoreload 2

from process_data import PubmedQuerys, process_TAR
import json

with open("./data/training-logs.jsonl", "w") as f:
    for item in PubmedQuerys().process_searchrefiner_logs():
        f.write(json.dumps(item) + "\n")

process_TAR()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


./data/2022-searchrefiner.log.jsonl: 5963it [00:00, 131016.38it/s]
./data/2024-searchrefiner.log.jsonl: 943it [00:00, 104788.15it/s]
3422it [00:00, 1711328.04it/s]
